# SenSwiss 2021

Code to run in parallel with the acquisition of images in SpinView.

Instructions: 

* Check that the **exposure time** of the camera is constant 11113us
* Check you have a folder named `/focus` at the main path
* Create a **new folder for the measurement** `/images/NEW_FOLDER_FOR_NEW_TEST` (this needs to be done each time, not in advance!!! (code checks last created folder))
* Follow the instructions below
* For acquisition in SpinView, save the images in the folder you created in step 3 as type .tiff
* If the chip moves at some point
* Look that everything is okay in the prints
* Press ctrl-j (for like 1-2seconds) when you want to finish computing the preprocessing and analysis and save the results in result.csv, which will be placed in the folder of step 3.

1. Set everything up in SpinView:
    * `/focus`
    * `/images/NEW_FOLDER_FOR_NEW_TEST`
    * exposure auto OFF, 11113us
    
2. Take an image for the focus before flowing BB (just in case) -> save as .tiff

3. Take another image if you moved the chip

4. Once you are sure the focus is the one you want, press **RUN in run_parallel** notebook

5. After, **start** the acquisition in **SpinView** (do not forget this step!!!)

## Integration with pump
1. connect with port
2. initialise
3. cleaning
4. pull BB
5. push pull sample
6. start recording SpinView
7. run parallel

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import keyboard
import sys
#import serial
import time

import run_parallel
#from pump import initialise_LSPone, general_cleaning_procedure, pick_BB
from run_parallel import describe_folders, press_ROIs, run_analysis

## 0. Pump

To connect the pump to the computer (only done at the very beginning of times).

In [ ]:
# Open serial connection -> check COM port on your device
lsp = serial.Serial('COM5', 9600, timeout=1000)  # TODO: change the comport depending on your computer
print('LSPone connected on ', lsp.name)

# Initialise LSPone normally already done
initialise_LSPone(lsp)

# 0. General Cleaning # this were i start
# 2: PBS, 3: Air, 4: Ethanol   (order of the tubes in the pump)
general_cleaning_procedure(lsp)

## Starting the measurements

#### 1. Take a photo with SpinView of the spots -> save at /focus

#### 2. Aspiring the Blocking Buffer

In [ ]:
# Sucking BB at 50 ul/min
pick_BB(lsp)

#### 3. Select ROI


In [11]:
# Select ROI
ORIGINAL_FOLDER, IMG_PROCESSED_FOLDER, IMG_FOLDER, DIR_ROI, DIR = describe_folders()
ROIs = press_ROIs(DIR_ROI, ORIGINAL_FOLDER)

THIS IS ORIGINAL FOLDER PATH /Users/janet/EnginyeriaBiomèdica/MSc LIFE SCIENCES ENGINEERING/SensUs/Code/SensUs_2021_Code
/Users/janet/EnginyeriaBiomèdica/MSc LIFE SCIENCES ENGINEERING/SensUs/Code/SensUs_2021_Code/focus
['test.tiff', 'test2.tiff', 'test2 còpia.tiff', '.gitkeep', 'test2 còpia 2.tiff', 'yesyesyes.tiff', '.DS_Store', 'yesyesyes còpia.tiff', '10ng_used_chip-08292021192426-299.tiff']
ROI PATH /Users/janet/EnginyeriaBiomèdica/MSc LIFE SCIENCES ENGINEERING/SensUs/Code/SensUs_2021_Code/focus/10ng_used_chip-08292021192426-299.tiff
Select the ROI. Press right button if you want to delete. The last 2 ROIs will be used as background. Press 'q' when you have finished. 
ROIs:
 [[3760 1708  480]
 [1160 2396  480]
 [1276  644  480]
 [2848 2772  480]]


#### 4. Start running preprocess and analysis

In [ ]:
# Run processing and analysis
run_analysis(ROIs, IMG_FOLDER=IMG_FOLDER, DIR=DIR, window_size=5, framerate=2, threshold=140)

OBSERVED FOLDER /Users/janet/EnginyeriaBiomèdica/MSc LIFE SCIENCES ENGINEERING/SensUs/Code/SensUs_2021_Code/images/finding_thresh
Press "s" or ctrl+c to exit and save the results
filename /Users/janet/EnginyeriaBiomèdica/MSc LIFE SCIENCES ENGINEERING/SensUs/Code/SensUs_2021_Code/images/finding_thresh/10ng_used_chip-08292021191415-0.tiff


Exception in thread Thread-8:
Traceback (most recent call last):
  File "/Applications/anaconda3/lib/python3.7/site-packages/skimage/external/tifffile/tifffile.py", line 1363, in _fromfile
    self.byteorder = {b'II': '<', b'MM': '>'}[self._fh.read(2)]
KeyError: b''

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Applications/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Applications/anaconda3/lib/python3.7/site-packages/watchdog/observers/api.py", line 199, in run
    self.dispatch_events(self.event_queue, self.timeout)
  File "/Applications/anaconda3/lib/python3.7/site-packages/watchdog/observers/api.py", line 372, in dispatch_events
    handler.dispatch(event)
  File "/Applications/anaconda3/lib/python3.7/site-packages/watchdog/events.py", line 278, in dispatch
    }[event.event_type](event)
  File "/Users/janet/EnginyeriaBiomèdica/MSc LIFE SCIENCES ENGINEERING/SensUs/Cod

#### 5. Flow the sample

In [ ]:
# Push and pull sample at 50 ul/min
push_pull_sample(lsp, n_times = 4)

#### 6. SpinView --> start recording

#### 7. View results (change name)

In [ ]:
# Loading results
result_df = pd.read_csv('./images/20210828/result.csv')   # TODO CHANGE NAME
result_df['Signal'].plot()

In [ ]:
# Running in parallel (if you want to run all at once)
# run_parallel.main()